<a href="https://colab.research.google.com/github/tomonari-masada/course2021-sml/blob/main/12_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tree-based methodsによる住宅価格の予測

* 前に使ったCalifornia housing datasetを使う。

 * scikit-learnからロードできるバージョンは、前処理が済んだキレイなデータなので、ここでは使わない。

* データの取得や前処理の一部は、
[Aurélien Géron. Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition.](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) の2章と同じ。

In [ ]:
import numpy as np
from scipy import stats, special
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score

%config InlineBackend.figure_format = 'retina'

## 1) データを取得

In [ ]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
  os.makedirs(housing_path, exist_ok=True)
  tgz_path = os.path.join(housing_path, "housing.tgz")
  urllib.request.urlretrieve(housing_url, tgz_path)
  housing_tgz = tarfile.open(tgz_path)
  housing_tgz.extractall(path=housing_path)
  housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
def load_housing_data(housing_path=HOUSING_PATH):
  csv_path = os.path.join(housing_path, "housing.csv")
  return pd.read_csv(csv_path)

（ここより上の詳細はフォローしなくてもいいいです。）

In [ ]:
housing = load_housing_data()
housing.head()

## 2) データを概観しつつ前処理

In [ ]:
housing.info()

* 数値データではない列が一つだけある



In [ ]:
housing['ocean_proximity'].value_counts()

* 今回は、この数値データではない列を残す
 * pandasのget_dummiesを使う



In [ ]:
housing_num = pd.get_dummies(housing, columns=['ocean_proximity'])

In [ ]:
housing_num.info()

* Non-Null Countが他より少ない列がある
 * ここでは単に削除することにする



In [ ]:
housing_num = housing_num.dropna(subset=['total_bedrooms'])

In [ ]:
housing_num.info()

* 説明変数と目的変数を分ける



In [ ]:
X = housing_num.drop('median_house_value', axis=1)
y = housing_num["median_house_value"].copy()

## 3) 評価実験のための準備

* 今回は交差検証をおこなうので、テストデータだけを切り分けておく。

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 4) 前処理

* どのような前処理をおこなうかは、お任せします。

In [ ]:
X_train.hist(bins=50, figsize=(15,12))

## 5) 決定木をチューニング

* 木の深さ（領域を何分割するか）をチューニングする。

* cross_val_scoreで使える評価尺度一覧をチェックする。
 * いずれも、「値が大きいほど良い」という評価尺度になっている。
 * 今回はRMSEで評価したいが、これは「値が小さいほど良い」という評価尺度である。
 * そこで、一覧の中にある'neg_root_mean_squared_error'に、マイナスをつけたものを、使う。
 * これで前の結果と比較できるようになる。



In [ ]:
sorted(metrics.SCORERS.keys())

* 10-fold cross validationを実施
 * 木の深さは3から20まで変える。
 * 評価尺度は　RMSEとする。



In [ ]:
for i in range(3, 16):
  reg = tree.DecisionTreeRegressor(max_depth=i, random_state=123)
  reg.fit(X_train, y_train)
  scores = cross_val_score(estimator=reg, X=X_train, y=y_train,
                           scoring='neg_root_mean_squared_error', cv=10)
  print(f'depth {i}: {- scores.mean():.2f}') 

* 最適な木の深さを使って訓練データ全体で学習をやり直し、テストデータで評価。



In [ ]:
reg = tree.DecisionTreeRegressor(max_depth=9, random_state=123)
reg.fit(X_train, y_train)
y_test_pred = reg.predict(X_test)
print('Test set RMSE: {:.2f}'.format(mean_squared_error(y_test, y_test_pred, squared=False)))

* これが今回のベースラインです。
* この値を改善することを試みてください。

# 課題12

* RMSEによって評価される予測性能を、良くして下さい
* test setとそれ以外の部分の分割は、変えないでください
 * test set以外の部分をどう使うかは、自由です
 * 交差検証の方法は何でもよいです。
* 決定木ベースの手法やその系統の手法なら、何を使ってもいいです。
* test setでのRMSEによる評価は、最後に一回おこなうだけです

## A) XGBoostを使ってみる



In [ ]:
import xgboost as xgb

* XGBoostが出すエラーへの対処
 * https://stackoverflow.com/questions/48645846/pythons-xgoost-valueerrorfeature-names-may-not-contain-or

In [ ]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]
X_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_test.columns.values]

* 10-fold cross validationを実施


In [ ]:
for i in range(6, 14):
  reg = xgb.XGBRegressor(objective ='reg:squarederror', max_depth=i, random_state=123)
  scores = cross_val_score(estimator=reg, X=X_train, y=y_train, 
                           scoring='neg_root_mean_squared_error', cv=10)
  print(f'depth {i}: {- scores.mean():.2f}')

* 最適な木の深さを使って訓練データ全体で学習をやり直し、テストデータで評価。



In [ ]:
# 各自、実践してください。

## B) CatBoostを使ってみる

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
for i in range(6, 14):
  model = CatBoostRegressor(iterations=200, depth=i, random_seed=123, logging_level='Silent')
  scores = cross_val_score(estimator=model, X=X_train, y=y_train,
                           scoring='neg_root_mean_squared_error', cv=10)
  print(f'depth {i}: {- scores.mean():.2f}')

* 最適な木の深さを使って訓練データ全体で学習をやり直し、テストデータで評価。



In [ ]:
best_depth = 10
model = CatBoostRegressor(iterations=200, depth=best_depth, random_seed=123)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('Test set RMSE: {:.2f}'.format(mean_squared_error(y_test, y_test_pred, squared=False)))